In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/carboni/Downloads/Leopard.csv')

### Clusters and other keywords

In [ ]:
keyword = 'Bête'
df = df[df['Cluster'].str.contains(keyword, case=False, na=False)]

In [3]:
df.head()

,Cluster,manifest_url,canvas_number,image_url,City,Country,Title,wkt,Date,Journal Type,Unnamed: 10
0,Bête Seule (Tigre),https://iiif.unige.ch/dhportal/ug796777/manifest,26.0,https://iiif.unige.ch/iiif/2/visualcontagions/...,"Washington, D.C.",United States of America,National Geographic Society,POINT(-77.036666666667 38.895),2019-01-01,Geography,NaN
1,Fourrure / Habit Léopard,https://iiif.unige.ch/dhportal/ug802806/manifest,17.0,https://iiif.unige.ch/iiif/2/visualcontagions/...,"Washington, D.C.",United States of America,National Geographic Society,POINT(-77.036666666667 38.895),2018-01-01,Geography,NaN
2,Fourrure / Habit Léopard,https://iiif.unige.ch/dhportal/ug615338/manifest,74.0,https://iiif.unige.ch/iiif/2/visualcontagions/...,"Washington, D.C.",United States of America,National Geographic Society,POINT(-77.036666666667 38.895),2018-01-01,Geography,NaN
3,Fourrure / Habit Léopard,https://iiif.unige.ch/dhportal/ug771741/manifest,75.0,https://iiif.unige.ch/iiif/2/visualcontagions/...,"Washington, D.C.",United States of America,National Geographic Society,POINT(-77.036666666667 38.895),2018-01-01,Geography,NaN
4,Bête et Corps Léopard,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,54.0,https://digi.ub.uni-heidelberg.de/iiif/2/porta...,Gdańsk,Poland,Porta aurea : rocznik Instytutu Historii Sztuk...,POINT(18.655 54.348055555555554),2018-01-01,Art History,NaN


## Index city 

### If WKT

In [5]:
df[['lon', 'lat']] = df['wkt'].str.extract(r'POINT\(([^ ]+) ([^ ]+)\)')

### if not wkt

In [6]:
city_coords = df.groupby('City').agg({'lat': 'first', 'lon': 'first'}).reset_index()

In [7]:
city_coords.rename({'City': 'name'}, axis=1, inplace=True)

In [8]:
city_coords.index = city_coords.index + 1
city_coords.index.name = 'id'

In [9]:
print(city_coords)

                 name                lat                  lon
id                                                           
1            Aberdeen  46.97536111111111  -123.81572222222222
2         Albuquerque    35.116666666667     -106.61666666667
3             Algiers    36.776388888889      3.0586111111111
4                Alma    43.378888888889     -84.659722222222
5            Amarillo    35.199166666667     -101.84527777778
..                ...                ...                  ...
223        Wilmington    34.223333333333     -77.912222222222
224  Wisconsin Rapids    44.394444444444     -89.832777777778
225              Yale    43.130277777778     -82.797777777778
226            Évreux    49.023333333333               1.1525
227              Žiri          46.046019            14.108283

[227 rows x 3 columns]


In [20]:
city_coords.to_csv('od_city_index.csv', index=True)

## Index to OD Matrix

In [11]:
city_id_map = city_coords.reset_index().set_index('name')['id'].to_dict()

In [12]:
df['origin'] = df['City'].map(city_id_map).astype(int)
df['dest'] = df['City'].shift(-1).map(city_id_map).fillna(method='ffill').astype(int)
df.iloc[-1, df.columns.get_loc('dest')] = df.iloc[-1]['origin']

In [14]:
df.iloc[-1, df.columns.get_loc('dest')] = df.iloc[-1]['origin']

In [15]:
od_matrix = df.groupby(['origin', 'dest']).size().reset_index(name='count')

In [16]:
print(od_matrix)

     origin  dest  count
0         1   122      2
1         2     2      1
2         2    47      1
3         2    69      1
4         2   147      1
..      ...   ...    ...
857     223    75      1
858     224    14      1
859     225   122      1
860     226   147      1
861     227   114      1

[862 rows x 3 columns]


In [18]:
od_matrix.to_csv('od_matrix.csv', index=True)